<a href="https://colab.research.google.com/github/DaviAlbini/data-science-projects/blob/main/03-avancados/Clustering_on_birds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clusterização de Aves: Análise Exploratória com Dataset Birds (OpenML ID 41464)
A análise de Clusters caracteriza-se por ser uma técnica exploratória, de modo que não tem caráter preditivo para observações de fora da amostra. Se  novas observações forem adicionadas à amostra, novos agrupamentos devem ser realizados, pois a inclusão de novas observações pode alterar a composição dos grupos. Se forem alteradas variáveis da análise, novos agrupamentos devem ser realizados, pois a inclusão/retirada de uma variável pode alterar os grupos.

Analisaremos dois métodos para a obtenção de Clusters:

 • **Método Hierárquico Aglomerativo**:
A quantidade de clusters é definida ao longo da análise (passo a passo)

 • **Método Não Hierárquico *K-means*** :
Define-se a priori quantos cluster serão formados

### **Método Hierárquico Aglomerativo**

A análise de cluster hierárquica depende de escolhas:

1.   Medida de dissimilaridade (*distância*)

> Refere-se à distância entre as observações, com base nas variáveis escolhidas. Portanto, indica o quanto as observações são diferentes entre si.

2.   Método de encadeamento das observações

> Refere-se à especificação da medida de distância quando houver cluster formados.

---

**Método de encadeamento**:

Indica qual distância utilizar quando já existem clusters formados durante os estágios aglomerativos.

*   Nearest neighbor (*single linkage*)

> Privilegia menores distâncias, recomendável em casos de observações distintas.

*   Furthest neigbor (*complete linkage*)

> Privilegia maiores distâncias, recomendável em casos de observações parecidas.

*   Between groups (*average linkage*)

> Junção de grupos pela distância média entre todos os pares de observações do grupo em análise.

### **Método Não Hierárquico K-means**

A quantidade `K` de clusters é **escolhida a priori** e é usada como base para a identificação dos centros de aglomeração, de modo que as observações são **arbitrariamente** alocadas aos `K` clusters para o cálculo dos centroides iniciais.

Nas etapas seguintes, as observações vão sendo comparadas pela proximidade aos centroides dos outros clusters. Se houver realocação a outro cluster por estar mais próxima, os centroides são recalculados (em ambos os clusters). Trata-se de um **processo iterativo**.

Em resumo:
> Faz-se diversas simulações e realocações, até chegar numa solução ótima. Se entendermos que a solução ótima são 3 clusters, então o modelo buscará realizar a melhor alocação para estes 3 clusters.

O procedimento K-means encerra-se quando não for possível realocar qualquer observação por estar mais próxima do centroide de outro cluster:

O que indica que:

> A soma dos quadrados de cada observação até o centro do cluster alocada foi minimizada.

---

**Técnicas para a identificação da quantidade de clusters no K-means**

1.   Método de Elbow

> Calcula-se a soma total dos quadrados dentro dos clusters (WCSS) para várias opções de K (quantidade de clusters). No gráfico, busca-se a dobra (“cotovelo”), ou seja, o ponto a partir do qual a diminuição na WCSS não é mais tão expressiva, mesmo aumentando a quantidade de clusters

2.   Método da Silhueta

> Para cada observação, calcula-se: (a) sua distância média dentro do cluster onde está alocada; (b) sua distância média para o cluster mais próximo onde não esteja alocada. Em seguida, calcula-se o coeficiente de silhueta médio para todas as observações. O procedimento é realizado para várias opções de K.

### **Considerações Importantes sobre Clusterização**

*   A análise de cluster é bastante **sensível** à presença de `outliers`.
*   Quando **há variáveis categóricas**, pode-se aplicar a `Análise de Correspondência` --> **Evitando ponderações arbitrárias**.
*   O `output` do **método hierárquico** pode ser utilizado como `input` no **método não hierárquico** para a identificação inicial da quantidade de clusters.
*   O **método não hierárquico k-means** pode ser aplicado em amostras maiores.

 ## Visão Geral do Dataset
- **Fonte**: OpenML (ID 41464)  
- **Objeto**: 327 gravações de áudio, 12 espécies de aves  
- **Características**: possíveis múltiplas espécies por gravação => cenário multilabel :contentReference[oaicite:18]{index=18}

## Objetivo do Projeto
Alcançar uma clusterização eficaz das instâncias de áudio para:
- Identificar padrões acústicos compartilhados entre espécies
- Mapear agrupamentos a características biológicas
- Facilitar visualizações interpretativas

## Clustering

In [33]:
# !pip install openml

In [1]:
# Importação e Manipulação de Dados
import os
import numpy as np
import pandas as pd
import openml

# Visualização
import plotly.express as px
import plotly.graph_objects as go

# Pré-processamento
from scipy.stats import zscore
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Utilidade
from sklearn.utils import check_random_state

RANDOM_STATE = 42
rs = check_random_state(RANDOM_STATE)

# Exibir mais colunas no console
pd.set_option("display.max_columns", 300)

In [2]:
oml_ds = openml.datasets.get_dataset(41464)
X, y, categorical_indicator, attribute_names = oml_ds.get_data(dataset_format="dataframe", target=None)
df = X.copy()

df.describe()

,audio.ssd1,audio.ssd2,audio.ssd3,audio.ssd4,audio.ssd5,audio.ssd6,audio.ssd7,audio.ssd8,audio.ssd9,audio.ssd10,audio.ssd11,audio.ssd12,audio.ssd13,audio.ssd14,audio.ssd15,audio.ssd16,audio.ssd17,audio.ssd18,audio.ssd19,audio.ssd20,audio.ssd21,audio.ssd22,audio.ssd25,audio.ssd26,audio.ssd27,audio.ssd28,audio.ssd29,audio.ssd30,audio.ssd31,audio.ssd32,audio.ssd33,audio.ssd34,audio.ssd35,audio.ssd36,audio.ssd37,audio.ssd38,audio.ssd39,audio.ssd40,audio.ssd41,audio.ssd42,audio.ssd43,audio.ssd44,audio.ssd45,audio.ssd46,audio.ssd49,audio.ssd50,audio.ssd51,audio.ssd52,audio.ssd53,audio.ssd54,audio.ssd55,audio.ssd56,audio.ssd57,audio.ssd58,audio.ssd59,audio.ssd60,audio.ssd61,audio.ssd62,audio.ssd63,audio.ssd64,audio.ssd65,audio.ssd66,audio.ssd67,audio.ssd68,audio.ssd69,audio.ssd70,audio.ssd73,audio.ssd74,audio.ssd75,audio.ssd76,audio.ssd77,audio.ssd78,audio.ssd79,audio.ssd80,audio.ssd81,audio.ssd82,audio.ssd83,audio.ssd84,audio.ssd85,audio.ssd86,audio.ssd87,audio.ssd88,audio.ssd89,audio.ssd90,audio.ssd91,audio.ssd92,audio.ssd93,audio.ssd94,audio.ssd97,audio.ssd98,audio.ssd99,audio.ssd100,audio.ssd101,audio.ssd102,audio.ssd103,audio.ssd104,audio.ssd105,audio.ssd106,audio.ssd107,audio.ssd108,audio.ssd109,audio.ssd110,audio.ssd111,audio.ssd112,audio.ssd113,audio.ssd114,audio.ssd115,audio.ssd116,audio.ssd117,audio.ssd118,audio.ssd121,audio.ssd122,audio.ssd123,audio.ssd124,audio.ssd125,audio.ssd126,audio.ssd127,audio.ssd128,audio.ssd129,audio.ssd130,audio.ssd131,audio.ssd132,audio.ssd133,audio.ssd134,audio.ssd135,audio.ssd136,audio.ssd137,audio.ssd138,audio.ssd139,audio.ssd140,audio.ssd141,audio.ssd145,audio.ssd146,audio.ssd147,audio.ssd148,audio.ssd149,audio.ssd150,audio.ssd151,audio.ssd152,audio.ssd153,audio.ssd154,audio.ssd155,audio.ssd156,audio.ssd157,audio.ssd158,audio.ssd159,audio.ssd160,audio.ssd161,audio.ssd162,audio.ssd163,audio.ssd164,audio.ssd165,audio.ssd166,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10,cluster11,cluster12,cluster13,cluster14,cluster15,cluster16,cluster17,cluster18,cluster19,cluster20,cluster21,cluster22,cluster23,cluster24,cluster25,cluster26,cluster27,cluster28,cluster29,cluster30,cluster31,cluster32,cluster33,cluster34,cluster35,cluster36,cluster37,cluster38,cluster39,cluster40,cluster41,cluster42,cluster43,cluster44,cluster45,cluster46,cluster47,cluster48,cluster49,cluster50,cluster51,cluster52,cluster53,cluster54,cluster55,cluster56,cluster57,cluster59,cluster60,cluster61,cluster62,cluster63,cluster64,cluster65,cluster66,cluster67,cluster68,cluster69,cluster70,cluster71,cluster72,cluster73,cluster74,cluster75,cluster76,cluster78,cluster79,cluster80,cluster81,cluster82,cluster83,cluster84,cluster85,cluster86,cluster87,cluster88,cluster89,cluster90,cluster91,cluster92,cluster93,cluster94,cluster95,cluster96,cluster97,cluster98,cluster99,cluster100,segments,mean_rect_width,std_rect_width,mean_rect_height,std_rect_height,mean_rect_volume,std_rect_volume
count,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645.000000,645

In [3]:
# Colunas Numéricas
num_cols = []
num_cols += [c for c in df.columns if c.startswith("audio.ssd")]
num_cols += [c for c in df.columns if c.startswith("cluster")]
num_cols += [c for c in df.columns if c.startswith("mean_")]
num_cols += [c for c in df.columns if c.startswith("std_")]

# Colunas Categóricas
categ_cols = ["segments", "location"]
df[categ_cols] = df[categ_cols].astype(str)

**Tratamento inicial dos dados:**

Antes de iniciar os procedimentos, é importante realizar uma análise das
unidades de medidas das variáveis. Se estiverem em unidades de medidas distintas, é importante realizar a padronização das variáveis antes de iniciar a análise de cluster.

> Dentre as técnicas de padronização temos `ZScore`/`Scaler` (ambos tornam as variáveis com média = 0 e desvio padrão = 1)



In [4]:
# Aplicando o procedimento de ZScore
df[num_cols] = df[num_cols].apply(zscore, ddof=1)

# Transformando categóricas em float com `get dummies`
df_encoded = pd.get_dummies(df, columns=categ_cols, drop_first=False)

In [93]:
df_encoded

,audio.ssd1,audio.ssd2,audio.ssd3,audio.ssd4,audio.ssd5,audio.ssd6,audio.ssd7,audio.ssd8,audio.ssd9,audio.ssd10,audio.ssd11,audio.ssd12,audio.ssd13,audio.ssd14,audio.ssd15,audio.ssd16,audio.ssd17,audio.ssd18,audio.ssd19,audio.ssd20,audio.ssd21,audio.ssd22,audio.ssd25,audio.ssd26,audio.ssd27,audio.ssd28,audio.ssd29,audio.ssd30,audio.ssd31,audio.ssd32,audio.ssd33,audio.ssd34,audio.ssd35,audio.ssd36,audio.ssd37,audio.ssd38,audio.ssd39,audio.ssd40,audio.ssd41,audio.ssd42,audio.ssd43,audio.ssd44,audio.ssd45,audio.ssd46,audio.ssd49,audio.ssd50,audio.ssd51,audio.ssd52,audio.ssd53,audio.ssd54,audio.ssd55,audio.ssd56,audio.ssd57,audio.ssd58,audio.ssd59,audio.ssd60,audio.ssd61,audio.ssd62,audio.ssd63,audio.ssd64,audio.ssd65,audio.ssd66,audio.ssd67,audio.ssd68,audio.ssd69,audio.ssd70,audio.ssd73,audio.ssd74,audio.ssd75,audio.ssd76,audio.ssd77,audio.ssd78,audio.ssd79,audio.ssd80,audio.ssd81,audio.ssd82,audio.ssd83,audio.ssd84,audio.ssd85,audio.ssd86,audio.ssd87,audio.ssd88,audio.ssd89,audio.ssd90,audio.ssd91,audio.ssd92,audio.ssd93,audio.ssd94,audio.ssd97,audio.ssd98,audio.ssd99,audio.ssd100,audio.ssd101,audio.ssd102,audio.ssd103,audio.ssd104,audio.ssd105,audio.ssd106,audio.ssd107,audio.ssd108,audio.ssd109,audio.ssd110,audio.ssd111,audio.ssd112,audio.ssd113,audio.ssd114,audio.ssd115,audio.ssd116,audio.ssd117,audio.ssd118,audio.ssd121,audio.ssd122,audio.ssd123,audio.ssd124,audio.ssd125,audio.ssd126,audio.ssd127,audio.ssd128,audio.ssd129,audio.ssd130,audio.ssd131,audio.ssd132,audio.ssd133,audio.ssd134,audio.ssd135,audio.ssd136,audio.ssd137,audio.ssd138,audio.ssd139,audio.ssd140,audio.ssd141,audio.ssd145,audio.ssd146,audio.ssd147,audio.ssd148,audio.ssd149,audio.ssd150,audio.ssd151,audio.ssd152,audio.ssd153,audio.ssd154,audio.ssd155,audio.ssd156,audio.ssd157,audio.ssd158,audio.ssd159,audio.ssd160,audio.ssd161,audio.ssd162,audio.ssd163,...,cluster12,cluster13,cluster14,cluster15,cluster16,cluster17,cluster18,cluster19,cluster20,cluster21,cluster22,cluster23,cluster24,cluster25,cluster26,cluster27,cluster28,cluster29,cluster30,cluster31,cluster32,cluster33,cluster34,cluster35,cluster36,cluster37,cluster38,cluster39,cluster40,cluster41,cluster42,cluster43,cluster44,cluster45,cluster46,cluster47,cluster48,cluster49,cluster50,cluster51,cluster52,cluster53,cluster54,cluster55,cluster56,cluster57,cluster59,cluster60,cluster61,cluster62,cluster63,cluster64,cluster65,cluster66,cluster67,cluster68,cluster69,cluster70,cluster71,cluster72,cluster73,cluster74,cluster75,cluster76,cluster78,cluster79,cluster80,cluster81,cluster82,cluster83,cluster84,cluster85,cluster86,cluster87,cluster88,cluster89,cluster90,cluster91,cluster92,cluster93,cluster94,cluster95,cluster96,cluster97,cluster98,cluster99,cluster100,mean_rect_width,std_rect_width,mean_rect_height,std_rect_height,mean_rect_volume,std_rect_volume,hasSegments,Brown.Creeper,Pacific.Wren,Pacific.slope.Flycatcher,Red.breasted.Nuthatch,Dark.eyed.Junco,Olive.sided.Flycatcher,Hermit.Thrush,Chestnut.backed.Chickadee,Varied.Thrush,Hermit.Warbler,Swainson.s.Thrush,Hammond.s.Flycatcher,Western.Tanager,Black.headed.Grosbeak,Golden.Crowned.Kinglet,Warbling.Vireo,MacGillivray.s.Warbler,Stellar.s.Jay,Common.Nighthawk,segments_0,segments_1,segments_10,segments_11,segments_12,segments_13,segments_14,segments_15,segments_16,segments_17,segments_18,segments_19,segments_2,segments_20,segments_21,segments_23,segments_24,segments_3,segments_36,segments_4,segments_5,segments_6,segments_7,segments_8,segments_9,location_1,location_10,location_11,location_13,location_15,location_16,location_17,location_2,location_4,location_5,location_7,location_8
0,-0.488936,-0.218091,-0.095540,0.143470,0.177156,0.310669,0.373549,0.468729,0.497583,0.476236,0.465066,0.337596,0.385230,0.413819,0.613032,0.710882,0.564552,0.445294,0.383684,0.517700,0.111989,-0.331619,-0.373093,-0.245675,-0.166137,-0.021787,-0.000693,0.038219,0.044428,0.124804,0.292491,0.128366,0.121869,-0.031718,0.069924,-0.154259,-0.093461,-0.121783,-0.118655,-0.164950,-0.057003

In [5]:
bool_cols = []
bool_cols += [c for c in df_encoded.columns if c.startswith("segments_")]
bool_cols += [c for c in df_encoded.columns if c.startswith("hasSegments")]
bool_cols += [c for c in df_encoded.columns if c.startswith("location_")]
df_encoded[bool_cols] = df_encoded[bool_cols].astype(float)

print(df_encoded.dtypes)

audio.ssd1    float64
audio.ssd2    float64
audio.ssd3    float64
audio.ssd4    float64
audio.ssd5    float64
               ...   
location_2    float64
location_4    float64
location_5    float64
location_7    float64
location_8    float64
Length: 314, dtype: object


In [6]:
df_birds = df_encoded.drop(columns=[
    "Brown.Creeper",
    "Pacific.Wren",
    "Pacific.slope.Flycatcher",
    "Red.breasted.Nuthatch",
    "Dark.eyed.Junco",
    "Olive.sided.Flycatcher",
    "Hermit.Thrush",
    "Chestnut.backed.Chickadee",
    "Varied.Thrush",
    "Hermit.Warbler",
    "Swainson.s.Thrush",
    "Hammond.s.Flycatcher",
    "Western.Tanager",
    "Black.headed.Grosbeak",
    "Golden.Crowned.Kinglet",
    "Warbling.Vireo",
    "MacGillivray.s.Warbler",
    "Stellar.s.Jay",
    "Common.Nighthawk"])

# df_birds = df_encoded.copy()
df_birds

,audio.ssd1,audio.ssd2,audio.ssd3,audio.ssd4,audio.ssd5,audio.ssd6,audio.ssd7,audio.ssd8,audio.ssd9,audio.ssd10,audio.ssd11,audio.ssd12,audio.ssd13,audio.ssd14,audio.ssd15,audio.ssd16,audio.ssd17,audio.ssd18,audio.ssd19,audio.ssd20,audio.ssd21,audio.ssd22,audio.ssd25,audio.ssd26,audio.ssd27,audio.ssd28,audio.ssd29,audio.ssd30,audio.ssd31,audio.ssd32,audio.ssd33,audio.ssd34,audio.ssd35,audio.ssd36,audio.ssd37,audio.ssd38,audio.ssd39,audio.ssd40,audio.ssd41,audio.ssd42,audio.ssd43,audio.ssd44,audio.ssd45,audio.ssd46,audio.ssd49,audio.ssd50,audio.ssd51,audio.ssd52,audio.ssd53,audio.ssd54,audio.ssd55,audio.ssd56,audio.ssd57,audio.ssd58,audio.ssd59,audio.ssd60,audio.ssd61,audio.ssd62,audio.ssd63,audio.ssd64,audio.ssd65,audio.ssd66,audio.ssd67,audio.ssd68,audio.ssd69,audio.ssd70,audio.ssd73,audio.ssd74,audio.ssd75,audio.ssd76,audio.ssd77,audio.ssd78,audio.ssd79,audio.ssd80,audio.ssd81,audio.ssd82,audio.ssd83,audio.ssd84,audio.ssd85,audio.ssd86,audio.ssd87,audio.ssd88,audio.ssd89,audio.ssd90,audio.ssd91,audio.ssd92,audio.ssd93,audio.ssd94,audio.ssd97,audio.ssd98,audio.ssd99,audio.ssd100,audio.ssd101,audio.ssd102,audio.ssd103,audio.ssd104,audio.ssd105,audio.ssd106,audio.ssd107,audio.ssd108,audio.ssd109,audio.ssd110,audio.ssd111,audio.ssd112,audio.ssd113,audio.ssd114,audio.ssd115,audio.ssd116,audio.ssd117,audio.ssd118,audio.ssd121,audio.ssd122,audio.ssd123,audio.ssd124,audio.ssd125,audio.ssd126,audio.ssd127,audio.ssd128,audio.ssd129,audio.ssd130,audio.ssd131,audio.ssd132,audio.ssd133,audio.ssd134,audio.ssd135,audio.ssd136,audio.ssd137,audio.ssd138,audio.ssd139,audio.ssd140,audio.ssd141,audio.ssd145,audio.ssd146,audio.ssd147,audio.ssd148,audio.ssd149,audio.ssd150,audio.ssd151,audio.ssd152,audio.ssd153,audio.ssd154,audio.ssd155,audio.ssd156,audio.ssd157,audio.ssd158,audio.ssd159,audio.ssd160,audio.ssd161,audio.ssd162,audio.ssd163,audio.ssd164,audio.ssd165,audio.ssd166,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10,cluster11,cluster12,cluster13,cluster14,cluster15,cluster16,cluster17,cluster18,cluster19,cluster20,cluster21,cluster22,cluster23,cluster24,cluster25,cluster26,cluster27,cluster28,cluster29,cluster30,cluster31,cluster32,cluster33,cluster34,cluster35,cluster36,cluster37,cluster38,cluster39,cluster40,cluster41,cluster42,cluster43,cluster44,cluster45,cluster46,cluster47,cluster48,cluster49,cluster50,cluster51,cluster52,cluster53,cluster54,cluster55,cluster56,cluster57,cluster59,cluster60,cluster61,cluster62,cluster63,cluster64,cluster65,cluster66,cluster67,cluster68,cluster69,cluster70,cluster71,cluster72,cluster73,cluster74,cluster75,cluster76,cluster78,cluster79,cluster80,cluster81,cluster82,cluster83,cluster84,cluster85,cluster86,cluster87,cluster88,cluster89,cluster90,cluster91,cluster92,cluster93,cluster94,cluster95,cluster96,cluster97,cluster98,cluster99,cluster100,mean_rect_width,std_rect_width,mean_rect_height,std_rect_height,mean_rect_volume,std_rect_volume,hasSegments,segments_0,segments_1,segments_10,segments_11,segments_12,segments_13,segments_14,segments_15,segments_16,segments_17,segments_18,segments_19,segments_2,segments_20,segments_21,segments_23,segments_24,segments_3,segments_36,segments_4,segments_5,segments_6,segments_7,segments_8,segments_9,location_1,location_10,location_11,location_13,location_15,location_16,location_17,location_2,location_4,location_5,location_7,location_8
0,-0.488936,-0.218091,-0.095540,0.143470,0.177156,0.310669,0.373549,0.468729,0.497583,0.476236,0.465066,0.337596,0.385230,0.413819,0.613032,0.710882,0.564552,0.445294,0.383684,0.517700,0.111989,-0.331619,-0.373093,-0.245675,-0.166137,-0.021787,-0.000693,0.038219,0.044428,0.124804,0.292491,0.128366,0.121869,-0.031718,0.069924,-0.154259,-0.093461,-0.121783,-0.118655,-0.164950,-0.057003,0.495600,-0.118059,-0.287158,-0.111597,-0.661833,-0.541884,-0.723564,-0.737741,-0.422742,-0.662208,-0.443700,-0.434404,-0.363630,-0.407698,-0.453658,-0.381992,-0.525665,-0.225699,0.037483,0.449017,1.098585,1.123206

In [13]:
birds_features = df_birds.columns
birds_features

Index(['audio.ssd1', 'audio.ssd2', 'audio.ssd3', 'audio.ssd4', 'audio.ssd5',
       'audio.ssd6', 'audio.ssd7', 'audio.ssd8', 'audio.ssd9', 'audio.ssd10',
       ...
       'location_11', 'location_13', 'location_15', 'location_16',
       'location_17', 'location_2', 'location_4', 'location_5', 'location_7',
       'location_8'],
      dtype='object', length=295)

In [30]:
# --- 1. Selecionar colunas
audio_cols = [col for col in df_birds.columns if col.startswith("audio.ssd")]
cluster_cols = [col for col in df_birds.columns if col.startswith("cluster")]
mean_std_cols = [col for col in df_birds.columns if col.startswith("mean_")] + [col for col in df_birds.columns if col.startswith("std_")] + [df_birds["hasSegments"]]
segments_cols = [col for col in df_birds.columns if col.startswith("segments_")]
location_cols = [col for col in df_birds.columns if col.startswith("location_")]

# --- 2. Calcular correlação apenas entre audio_cols (linhas) e other_cols (colunas)
# corr = df_birds[location_cols + audio_cols].corr()
corr = df_birds.corr()

# Extrair apenas bloco desejado (linhas=audio, colunas=outras)
corr_block = corr.loc[location_cols, audio_cols]

# --- 3. Plotar heatmap
fig = px.imshow(
    # corr_block,
    corr,
    text_auto=".2f",
    aspect="auto",
    color_continuous_scale="RdBu_r",
    title="Correlação",
    zmin=-1,   # limite inferior fixo
    zmax=1     # limite superior fixo
)

fig.update_layout(
    # xaxis_title="Outras variáveis",
    # yaxis_title="Variáveis audio.ssd",
    width=1500,
    height=900
)

fig.show()


Após qualquer tipo de clusterização é interessante avaliar quais variáveis contribuíram ou não para aqueles agrupamentos. É importante comparar se a **variabilidade dentro do grupo é menor do que a variabilidade entre grupos** com base nas variáveis da análise.

Aplica-se um **teste F** para análise de variância:

> 𝑭 = 𝑽𝒂𝒓𝒊𝒂𝒃𝒊𝒍𝒊𝒅𝒂𝒅𝒆 𝒆𝒏𝒕𝒓𝒆 𝒈𝒓𝒖𝒑𝒐𝒔 / 𝑽𝒂𝒓𝒊𝒂𝒃𝒊𝒍𝒊𝒅𝒂𝒅𝒆 𝒅𝒆𝒏𝒕𝒓𝒐 𝒅𝒐𝒔 𝒈𝒓𝒖𝒑𝒐𝒔

É possível analisar quais variáveis mais contribuíram para a formação de pelo menos um dos clusters:

> Maiores valores da estatística F (*em conjunto com a significância*)

## Principais Tecnologias
- **Python**: `pandas`, `numpy`, `scikit-learn`, `matplotlib` / `seaborn`, *t-SNE* / *UMAP*
- **Leitura de .arff**: `scipy.io.arff` ou `liac-arff`
- (Opcional) **OpenML**: usar a API para baixar o dataset diretamente (`openml.datasets.get_dataset(41464)`)

## Conclusão Esperada
- Visualizações de clusters bem separadas e interpretáveis
- Relacionamento entre clusters e espécies (ou ausência disso)
- Insights sobre características acústicas compartilhadas
- Apontamento de próximas fases (p.ex. classificação multilabel, extração de atributos complexos, generalização para novos dados)